Parcourt train/P*/T*/.
Calcule le volume de chaque lésion avec SimpleITK.
(Optionnel) Compare avec un fichier CSV de référence venant du dataset.
Sauvegarde tout dans un CSV pour vérifier les différences.

In [ ]:
import os
import csv
import SimpleITK as sitk

# Dossier racine contenant tous les patients
BASE_DIR = "../../27919209/MSLesSegDataset/train"

# CSV clinique exporté depuis Numbers
REFERENCE_CSV = "../../27919209/MSLesSegDataset/info_dataset/clinical_data_well_formated.csv"

# Chargement des valeurs de référence :
# clé: (Patient, Timepoint) -> dict(lesion_number_ref, lesion_volume_mm3_ref)
ref_info = {}

with open(REFERENCE_CSV, newline="") as f_ref:
    # très important : delimiter=';'
    reader = csv.DictReader(f_ref, delimiter=';')
    for raw_row in reader:
        # on nettoie les clés et les valeurs (espaces, retours chariot)
        row = {k.strip(): (v.strip() if isinstance(v, str) else v)
               for k, v in raw_row.items()}

        patient   = row["Patient"]        # ex: "P1"
        timepoint = row["Timepoint"]      # ex: "T1"

        # Lesion Number : entier
        lesion_num_ref = int(row["Lesion Number"])

        # Lesion Volume : déjà en mm^3 dans ton fichier (décimal avec un point)
        # Si un jour tu as une virgule ici, décommente la ligne avec replace
        lesion_vol_str = row["Lesion Volume"]
        # lesion_vol_str = lesion_vol_str.replace(',', '.')
        lesion_vol_ref = float(lesion_vol_str)

        key = (patient, timepoint)
        ref_info[key] = {
            "lesion_number": lesion_num_ref,
            "lesion_volume_mm3": lesion_vol_ref
        }

# Fichier de sortie
output_csv = "lesion_volume_validation.csv"

with open(output_csv, mode="w", newline="") as f_out:
    writer = csv.writer(f_out)
    writer.writerow([
        "patient",
        "session",
        "n_lesions_sitk",
        "n_lesions_ref",
        "diff_n_lesions",
        "volume_mm3_sitk",
        "volume_mm3_ref",
        "diff_volume_mm3",
        "diff_volume_percent"
    ])

    # Boucle sur les patients (P1, P2, ...)
    for patient in sorted(os.listdir(BASE_DIR)):
        patient_dir = os.path.join(BASE_DIR, patient)
        if not os.path.isdir(patient_dir):
            continue

        # Boucle sur les sessions (T1, T2, T3, T4, ...)
        for session in sorted(os.listdir(patient_dir)):
            session_dir = os.path.join(patient_dir, session)
            if not os.path.isdir(session_dir):
                continue

            prefix = f"{patient}_{session}"
            mask_path = os.path.join(session_dir, f"{prefix}_MASK.nii.gz")

            if not os.path.exists(mask_path):
                print(f"[WARN] Pas de masque pour {patient} / {session}, on saute.")
                continue

            print(f"Validation volumes: {patient} / {session} ...")

            # Lecture du masque
            mask_img = sitk.ReadImage(mask_path)

            # Masque binaire : lésion = voxels > 0
            lesion_bin = mask_img > 0

            # Composantes connexes 3D : une lésion = un label (1,2,3,...)
            lesion_cc = sitk.ConnectedComponent(lesion_bin)

            # Stats pour obtenir le nombre de voxels par lésion
            stats = sitk.LabelStatisticsImageFilter()
            stats.Execute(lesion_cc, lesion_cc)

            # Volume d'un voxel (en mm^3)
            sx, sy, sz = mask_img.GetSpacing()
            voxel_vol_mm3 = sx * sy * sz

            # Nombre de lésions et volume total
            n_lesions_sitk = 0
            total_voxels = 0

            for label in stats.GetLabels():
                if label == 0:
                    continue  # 0 = fond
                n_lesions_sitk += 1
                total_voxels += stats.GetCount(label)

            volume_mm3_sitk = total_voxels * voxel_vol_mm3

            # Récupération des valeurs de référence
            key = (patient, session)
            if key in ref_info:
                n_lesions_ref = ref_info[key]["lesion_number"]
                volume_mm3_ref = ref_info[key]["lesion_volume_mm3"]

                diff_n_lesions = n_lesions_sitk - n_lesions_ref
                diff_volume_mm3 = volume_mm3_sitk - volume_mm3_ref
                if volume_mm3_ref != 0:
                    diff_volume_percent = 100.0 * diff_volume_mm3 / volume_mm3_ref
                else:
                    diff_volume_percent = float("nan")
            else:
                n_lesions_ref = ""
                volume_mm3_ref = ""
                diff_n_lesions = ""
                diff_volume_mm3 = ""
                diff_volume_percent = ""

            writer.writerow([
                patient,
                session,
                n_lesions_sitk,
                n_lesions_ref,
                diff_n_lesions,
                volume_mm3_sitk,
                volume_mm3_ref,
                diff_volume_mm3,
                diff_volume_percent
            ])

print(f"Terminé. Résultats sauvegardés dans {output_csv}")


Traitement volumes: P1 / T1 ...
Traitement volumes: P1 / T2 ...
Traitement volumes: P1 / T3 ...
Traitement volumes: P10 / T1 ...
Traitement volumes: P10 / T2 ...
Traitement volumes: P11 / T1 ...
Traitement volumes: P11 / T2 ...
Traitement volumes: P12 / T1 ...
Traitement volumes: P12 / T2 ...
Traitement volumes: P12 / T3 ...
Traitement volumes: P12 / T4 ...
Traitement volumes: P13 / T1 ...
Traitement volumes: P13 / T2 ...
Traitement volumes: P14 / T1 ...
Traitement volumes: P14 / T2 ...
Traitement volumes: P14 / T3 ...
Traitement volumes: P14 / T4 ...
Traitement volumes: P15 / T1 ...
Traitement volumes: P16 / T1 ...
Traitement volumes: P17 / T1 ...
Traitement volumes: P18 / T1 ...
Traitement volumes: P19 / T1 ...
Traitement volumes: P19 / T2 ...
Traitement volumes: P19 / T3 ...
Traitement volumes: P19 / T4 ...
Traitement volumes: P2 / T1 ...
Traitement volumes: P2 / T2 ...
Traitement volumes: P2 / T3 ...
Traitement volumes: P2 / T4 ...
Traitement volumes: P20 / T1 ...
Traitement volume